In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print('success!')

     |████████████████████████████████| 993kB 3.4MB/s 
success!


In [0]:
from progressbar import ProgressBar, Percentage, Bar
from google.colab import files
import torch, pickle, os, sys, random, time
from torch import nn, optim
from queue import PriorityQueue
import operator

In [0]:
#german = drive.CreateFile({'id': '17atuQ5p0LNY7nj4dcmlaFXD94dQV3NkB'}) all
#german = drive.CreateFile({'id': '1bNBrVR9TCNXRqlS9kxPa3EffYyZnAfCw'}) 50k
german = drive.CreateFile({'id': '1FuoqjfMofNfkIRXBIWDU_cIVV9BBWcs-'}) # with unk
german.GetContentFile('./german_unk.pickle') 
with open('./german_unk.pickle', 'rb') as f_in:
    german = pickle.load(f_in)

#english = drive.CreateFile({'id': '1NEoxrv_6GeoM0fPrpw4RWiyO19FaZd20'}) # all
english = drive.CreateFile({'id': '18Q-WRZ2JBwkAJLXwiVuWPfyE-0k76FDA'}) # with unk
#english = drive.CreateFile({'id': '1XnCWgSLGcp3lCdUK0ciS5m6mwI5L5R2l'}) # 50k
english.GetContentFile('./english_unk.pickle') 
with open('./english_unk.pickle', 'rb') as f_in:
    english = pickle.load(f_in)

for i in range(len(german['train'])):
    german['train'][i] = torch.LongTensor(german['train'][i]).cuda()
    english['train'][i] = torch.LongTensor(english['train'][i]).cuda()
    
for i in range(len(german['dev'])):
    german['dev'][i] = torch.LongTensor(german['dev'][i]).cuda()
    english['dev'][i] = torch.LongTensor(english['dev'][i]).cuda()


In [0]:
def get_batches(german, english, b_sz):
    de_batches = [[]]
    
    # For every sentence in the dataset, add it to a batch, based on the batch size
    # if the sentence + current length is not greater than the batch size, 
    # then add it to the batch otherwise fill the current batch
    for sent in german:
        cur_len = 0
        for b in de_batches[-1]:
            cur_len += len(b)
    
        if (cur_len + len(sent)) <= b_sz: 
            de_batches[-1].append(sent)
        else:
            de_batches.append([])        
            de_batches[-1].append(sent)
    
    # For every batch within the entire set of batches, add padding to the sentences
    # that are less than the length of the longest sentence within the each batch.
    for b in de_batches:
        max_len = len(max(b, key=len))
        
        for sent in b:
            dif = max_len - len(sent)
            if dif > 0:
                pad_list = [0] * dif
                pad_list = torch.as_tensor(pad_list).to(device)
                sent.data = torch.cat((sent,pad_list), dim=0)

    # Within each batch, load the english sentence that corresponds to each
    # german sentence within that batch to a new english batch.
    en_batches = []
    k=0
    for i in range(len(de_batches)):
        tmp_batch = [0]*len(de_batches[i])
        for j in range(len(de_batches[i])):
            tmp_batch[j] = english[k]
            k+=1
        en_batches.append(tmp_batch)
        
    # Within each english batch, pad all the sentences to be the same length as the 
    # maximum sentence within that batch.
    for b in en_batches:
        max_len = len(max(b, key=len))
        
        for sent in b:
            dif = max_len - len(sent)
            if dif > 0:
                pad_list = [0] * dif
                pad_list = torch.as_tensor(pad_list).to(device)
                sent.data = torch.cat((sent,pad_list), dim=0)
        
    # Define the batches list, which contains an index for each batch. Within each index
    # of the batches list is a batch. Each batch is of length N (number of sentences), 
    # Each index (0..N) within the batch is a dictionary containing the source and target
    # sentence. Therefore, the batches list is a list of lists (essentially a 2D array), 
    # where each element in the list of lists/array is a dictionary containing the source
    # and target tensor.
    batches = []
    for i in range(len(de_batches)):
        dict_batch = []
        for j in range(len(de_batches[i])):
            tmp_dict = {"source": de_batches[i][j],
                       "target": en_batches[i][j]}
            dict_batch.append(tmp_dict)
        batches.append(dict_batch)
        
    return batches

In [0]:
def batchify(batch): # We need the batches to be of shape [sentence length, batch size]
    source_sent_len = (len(batch[0]['source']))
    target_sent_len = (len(batch[0]['target']))
    batch_size = (len(batch))
    
    source = torch.empty((source_sent_len, batch_size)).int().to(device)    
    target = torch.empty((target_sent_len, batch_size)).int().to(device)
    
    for i in range(len(batch)):
        source[:,i] = batch[i]['source']
        target[:,i] = batch[i]['target']
    
    return source, target


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        
        #outputs = [src sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden


class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs, beam=False):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
            
        #repeat encoder hidden state src_len times
        
        if not beam:
            hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        if not self.training and beam: #for beam search do this::::::
            hidden = hidden.repeat(encoder_outputs.size(0),encoder_outputs.size(1), 1)

        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #attention= [batch size, src len]
        
        return torch.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, beam=False):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        if not beam: 
            input = input.unsqueeze(0)
        
        #input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, beam)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim=2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim=1))
        
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0)

In [0]:

class Base(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.softmax = nn.LogSoftmax()
        
          
    def beam_search(self, target_tensor, decoder_hiddens, encoder_outputs=None):
        '''
        :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
        :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
        :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
        :return: decoded_batch
        '''

        beam_width = 10
        topk = 10 # how many sentence do you want to generate
        decoded_batch = []
        # decoding goes sentence by sentence
        sentence_length, num_sentences = target_tensor.size()
        for idx in range(num_sentences):
            if isinstance(decoder_hiddens, tuple):  # LSTM case
                decoder_hidden = (decoder_hiddens[0][:,idx, :].unsqueeze(0),decoder_hiddens[1][:,idx, :].unsqueeze(0))
            else:
                decoder_hidden = decoder_hiddens[idx, :].unsqueeze(0)
            encoder_output = encoder_outputs[:,idx, :].unsqueeze(1)

            # Start with the start of the sentence token
            decoder_input = torch.LongTensor([[2]]).cuda()

            # Number of sentence to generate
            endnodes = []
            number_required = min((topk + 1), topk - len(endnodes))

            # starting node -  hidden vector, previous node, word id, logp, length
            node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
            nodes = PriorityQueue()

            # start the queue
            nodes.put(node)
            qsize = 1

            # start beam search
            while True:
                # give up when decoding takes too long
                if qsize > 10000: break

                # fetch the best node
                n = nodes.get()
                decoder_input = n.wordid
                decoder_hidden = n.h

                if n.wordid.item() == 3 and n.prevNode != None:
                    endnodes.append(n)
                    # if we reached maximum # of sentences required
                    if len(endnodes) >= number_required:
                        break
                    else:
                        continue
                
                # decode for one step using decoder
                decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_output, True)

                # PUT HERE REAL BEAM SEARCH OF TOP
                # take softmax first, then log
                decoder_output = self.softmax(decoder_output)
                
                log_prob, indexes = torch.topk(decoder_output, beam_width)
                nextnodes = []

                for new_k in range(beam_width):
                    decoded_t = indexes[0][new_k].view(1, -1)
                    log_p = log_prob[0][new_k].item()

                    node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp - log_p, n.leng + 1)
                    score = -node.eval()
                    nextnodes.append(node)

                # put them into queue
                for i in range(len(nextnodes)):
                    nn = nextnodes[i]
                    nodes.put(nn)
                    # increase qsize
                qsize += len(nextnodes) - 1

            # choose nbest paths, back trace them
            if len(endnodes) == 0:
                endnodes = [nodes.get() for _ in range(topk)]

            utterances = []
            max_prob = -100000
            max_sen = False
            final_sentence = []
            for n in sorted(endnodes):
                utterance = []
                if max_prob < n.logp:
                    max_prob = n.logp
                    max_sen = True
                utterance.append(n.wordid)
                # back trace
                while n.prevNode != None:
                    n = n.prevNode
                    utterance.append(n.wordid)
                  
                utterance = utterance[::-1]
                if max_sen:
                    max_sen = False
                    final_sentence = utterance
                utterances.append(utterance)
                
                
            decoded_batch.append(torch.IntTensor(final_sentence))

        return decoded_batch
  
  
    def forward(self, src, trg, teacher_forcing_ratio=0.5, run_beam=False):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        #outputs = torch.zeros(max_len, batch_size, trg_vocab_size).cuda()
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden https://www.programiz.com/python-programming/methods/built-in/sortedis the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        output = trg[0,:]
        #return self._beam_search(encoder_outputs, hidden, output, max_len, 10)
      
        # beam search evaluation
        if run_beam:
            temp = self.beam_search(trg, hidden ,encoder_outputs)
            return temp
        # else use greedy
        
        decoded_batch = torch.zeros(max_len, batch_size, trg_vocab_size).cuda()
        decoder_input = trg[0,:]
        for t in range(0, max_len):
            decoder_output, hidden = self.decoder(decoder_input, hidden, encoder_outputs, run_beam)
            decoded_batch[t] = decoder_output
            topv, topi = decoder_output.data.topk(1)  # get candidates
            topi = topi.view(-1)
            decoder_input = topi
          
        return decoded_batch
      
      
class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        reward = 0
        # Add here a function for shaping a reward

        return self.logp / float(self.leng - 1 + 1e-6) + alpha * reward
      
      
    def __lt__(self, other):
        return self.logp < other.logp


In [0]:
def create_model(english_data, german_data, device):    
    INPUT_DIM = len(german_data['idx2word'])
    OUTPUT_DIM = len(english_data['idx2word'])
    ENC_EMB_DIM = 256
    DEC_EMB_DIM = 256
    ENC_HID_DIM = 512
    DEC_HID_DIM = 512
    ENC_DROPOUT = 0.5
    DEC_DROPOUT = 0.5
    attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM,
                  DEC_HID_DIM, ENC_DROPOUT)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM,
                  DEC_HID_DIM, DEC_DROPOUT, attn)
    model = Base(enc, dec, device).to(device)
    return model

In [0]:
def write_to_drive(filename):
    from googleapiclient.discovery import build
    drive_service = build('drive', 'v3')

    from googleapiclient.http import MediaFileUpload

    file_metadata = {
      'name': filename
    }
    media = MediaFileUpload(filename, 
                            resumable=True)
    created = drive_service.files().create(body=file_metadata,
                                           media_body=media,
                                           fields='id').execute()
    print('File ID: {}'.format(created.get('id')))

In [0]:
def run_model(net, params):
  
    print('batching starting')
    tr_batches = get_batches(german['train'], english['train'], b_sz=params['batch_size'])
    dev_batches =  get_batches(german['dev'], english['dev'], b_sz=params['batch_size'])
    
    print('batching done')
    
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    
    for epoch in range(params['epochs']):
        start_time = time.time()

        train_loss, checkpoint = train(net, criterion, optimizer, tr_batches)

        valid_loss = evaluate(net, criterion, dev_batches)

        end_time = time.time()

        print("Epoch 0 : ", end_time-start_time)
        print("Train_Loss : ", train_loss)
        print("Validation_Loss : ", valid_loss)
        
        checkpoint = {}
        checkpoint['state_dict'] = net.state_dict()
        checkpoint['optimizer'] = optimizer.state_dict()
        checkpoint['epoch'] = epoch
        file_name = 'checkpoint_{0}.pth'.format(epoch)        
        torch.save(checkpoint, file_name)
        write_to_drive(file_name)

In [0]:
def train(net, criterion, optimizer, tr_batches):
          
    net.train()
    
    random.shuffle(tr_batches)
    ep_loss = 0.

    pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=len(tr_batches))       
    # for each batch, calculate loss and optimize model parameters            
    i = 0
    pbar.start()
    for b in tr_batches: # TRAINING LOOP
        source, target = batchify(b)
        x,y = target.size()
        preds = net(source.long(), target.long())
        preds = preds[0:].view(-1, preds.shape[-1])
        eng_target = target[0:].view(-1)
        loss = criterion(preds, eng_target.long())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        ep_loss += loss/y
        pbar.update(i+1)
        i += 1
    pbar.finish()
    return ep_loss, checkpoint
  
  
def evaluate(net, criterion, dev_batches):
  
    total_loss = 0
    net.eval()    
    with torch.no_grad():
        
        for b in dev_batches:
            source, target = batchify(b)
            output = net(source.long(), target.long(), 0) #turn off teacher forcing
            x,y = target.size()
            output = output[0:].view(-1, output.shape[-1])
            target = target[0:].view(-1)



            loss = criterion(output, target.long())

            total_loss += loss/y
        
    return total_loss

In [0]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#     checkpoint = torch.load('checkpoint.pth')
#     net.load_state_dict(checkpoint['state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer'])
model = create_model(english, german, device)
params = {}
params['batch_size'] = 700
params['epochs'] = 7
params['learning_rate'] = 0.001
run_model(model, params)
#train(english['train'], english['dev'],german['train'], german['dev'], model, params)

batching starting


N/A%|                                                                         |

batching done


 27%|####################                                                     |

In [0]:

        
d = drive.CreateFile({'id': '1-VvPpxCta6jNeoOup_jb6hNugsTNIC0e'})
d.GetContentFile('./checkpoint.pth') 


In [0]:
#checkpoint = torch.load('checkpoint.pth')
torch.cuda.empty_cache()
#eva_batches = get_batches(german['dev'], english['dev'], b_sz=1000)
a = get_target(model)


batch num:  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


batch num:  1
batch num:  2
batch num:  3
batch num:  4
batch num:  5
batch num:  6
batch num:  7
batch num:  8
batch num:  9
batch num:  10
batch num:  11
batch num:  12


In [0]:
def get_target(net):
    net.load_state_dict(checkpoint['state_dict'])
    ep_loss = 0
    i = 0
    all_preds = []
    net.eval()
    
    for b in eva_batches: # TRAINING LOOP
        print('batch num: ', i)
        
        source, target = batchify(b)
        preds = net(source.long(), target.long(), 0, run_beam=True)
        eng_target = target[0:].view(-1)
        all_preds.append(preds)
        i += 1
    return all_preds
   

In [0]:
eng = []
for batch in a:
    for line in batch:
        sentence = ""
        for character in line[1:-1]:
            sentence += " " + english['idx2word'][character.item()]
        eng.append(sentence.strip())


In [0]:
eng_t = []
for j in range(len(eva_batches)):
  _, target = batchify(eva_batches[j])
  _, b = target.size()
  for i in range(b):
      sentence = []
      for idx in target[:,i]:
          sentence.append(english['idx2word'][idx.item()])
          if idx.item() == 3:
             break
      eng_t.append((' '.join(sentence[1:-1])))


In [0]:

with open('model.out', 'w') as f:
    for line in eng:
        f.write(line + '\n')



with open('real.out', 'w') as f:
    for line in eng_t[:len(eng)]:
        f.write(line + '\n')

In [0]:
!git clone https://github.com/moses-smt/mosesdecoder.git
!pip install sacrebleu

fatal: destination path 'mosesdecoder' already exists and is not an empty directory.


In [0]:
%%shell
#!/bin/bash

# This is a reference to the gold translations from the dev set
REFERENCE_FILE="real.out"

# XXX: Change the following line to point to your model's output!
TRANSLATED_FILE="model.out"

# The model output is expected to be in a tokenized form. Note, that if you
# tokenized your inputs to the model, then simply joined each output token with
# whitespace you should get tokenized outputs from your model.
# i.e. each output token is separate by whitespace
# e.g. "My model 's output is interesting ."
perl "mosesdecoder/scripts/tokenizer/detokenizer.perl" -l en < "$TRANSLATED_FILE" > "$TRANSLATED_FILE.detok"

PARAMS=("-tok" "intl" "-l" "de-en" "$REFERENCE_FILE")
sacrebleu "${PARAMS[@]}" < "$TRANSLATED_FILE.detok"

Detokenizer Version $Revision: 4134 $
Language: en
BLEU+case.mixed+lang.de-en+numrefs.1+smooth.exp+tok.intl+version.1.3.2 = 0.4 60.2/27.8/15.1/8.8 (BP = 0.017 ratio = 0.198 hyp_len = 32227 ref_len = 163089)


In [0]:
for i in range(len(eng)):
    print(eng_t[i], eng[i])

thank you . thank you .
thank you . thank you .
hello ! hello !
thank you . thank you .
okay . okay .
a hundred and forty-seven . ahhh .
thank you . thank you .
okay . okay .
thank you . thank you .
no . no .
thank you . thank you .
thanks . thank you .
yeah . right .
ole ! whoa !
thank you . thank you .
thank you . thank you .
ohhhh . awww .
absolutely . right .
obi-wan . poignant .
thank you . thank you .
thank you . thank you .
thank you . thank you .
thank you . thank you .
thank you . thank you .
ta-da ! whoa !
yeah . yes .
raw data . whoa .
sort of . cheers .
simple . simple .
awesome . fantastic .
wooo ! whoa !
thank you . thank you .
thank you . thank you .
yeah . yes .
fine . camelot .
creativity . creativity creativity .
whoa ! whoa !
thanks . thank you .
thank you . thank you .
thank you . thank you .
unique . unique .
thank you . thank you .
thank you . thank you .
thank you . thank you .
really . cheers .
yeah . yes !
all right . good .
thank you . thank you .
thank you . 